In [16]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Load your data
df_pars_train = pd.read_csv('Trainset_features_10000.csv')
df_pars_test = pd.read_csv('Testset_features.csv')

pars_train = np.array(df_pars_train)
pars_test = np.array(df_pars_test)

X = pars_train[:, 3:]
y = pars_train[:, 2]

X_testset = pars_test[:, 2:]

# Define the number of folds (k)
k = 11  # You can change this to your desired number of folds

# Initialize a stratified k-fold cross-validator
kf = StratifiedKFold(n_splits=k, shuffle=True, random_state=0)

# Initialize lists to store the evaluation results for each fold
accuracy_scores = []

for train_index, val_index in kf.split(X, y):
    X_train_fold, X_val_fold = X[train_index], X[val_index]
    y_train_fold, y_val_fold = y[train_index], y[val_index]

    # Standardize the data
    sc = StandardScaler()
    sc.fit(X_train_fold)
    X_train_std_fold = sc.transform(X_train_fold)
    X_val_std_fold = sc.transform(X_val_fold)
    X_testset_std = sc.transform(X_testset)

    # Create and compile the model
    ann = Sequential()
    ann.add(Dense(units=11, activation=tf.nn.leaky_relu))
    ann.add(Dense(units=128))
    ann.add(tf.keras.layers.PReLU())
    ann.add(Dense(units=32))
    ann.add(tf.keras.layers.PReLU())
    ann.add(Dense(units=1, activation="sigmoid"))
    ann.compile(optimizer="adam", loss="binary_crossentropy", metrics=['accuracy'])

    # Train the model
    ann.fit(X_train_std_fold, y_train_fold, batch_size=32, epochs=30, verbose=0)

    # Evaluate the model on the validation set
    y_pred_fold = ann.predict(X_val_std_fold)
    y_pred_fold = np.where(y_pred_fold > 0.5, 1, 0)
    accuracy = accuracy_score(y_val_fold, y_pred_fold)
    accuracy_scores.append(accuracy)

# Calculate the mean accuracy across all folds
mean_accuracy = np.mean(accuracy_scores)

# Print the mean accuracy
print("Mean Accuracy:", mean_accuracy)


57/57 [==============================] - 0s 828us/step
Mean Accuracy: 0.8369000451447014


In [17]:
y_pred_ANN =ann.predict(X_testset_std)
df_y_pred_ANN = pd.DataFrame(y_pred_ANN)
df_y_pred_ANN.to_csv('Test_Submission_10000_ANN_0930_11.csv')

63/63 [==============================] - 0s 565us/step


In [ ]:
## for k = 5, accuarcy 0.8385
## for k = 6, accuarcy 0.8395
## for k = 7, accuarcy 0.8405
## for k = 8, accuarcy 0.839
## for k = 9, accuarcy 0.8387
## for k = 10, accuarcy 0.8391